# Fonction calcul métriques WER CER à partir des jsonl

In [9]:
import json
from pathlib import Path
from jiwer import wer, cer

def evaluate_asr_predictions(jsonl_path: str):
    """
    Calcule WER et CER en comparant `raw_text` et `model_prediction` dans un fichier JSONL.
    Retourne un dictionnaire avec les métriques globales.
    """
    jsonl_path = Path(jsonl_path)
    if not jsonl_path.exists():
        raise FileNotFoundError(f"{jsonl_path} n'existe pas")

    references = []
    hypotheses = []

    with open(jsonl_path, "r", encoding="utf-8") as fin:
        for line in fin:
            item = json.loads(line)
            ref = item.get("raw_text", "").strip()
            hyp = item.get("model_prediction", "").strip()
            if ref and hyp:
                references.append(ref)
                hypotheses.append(hyp)

    if not references:
        raise ValueError("Aucune référence valide trouvée dans le fichier.")

    overall_wer = wer(references, hypotheses)
    overall_cer = cer(references, hypotheses)

    print(f"✅ Fichier évalué : {jsonl_path}")
    print(f"Nombre de phrases évaluées : {len(references)}")
    print(f"WER (Word Error Rate) : {overall_wer:.3f}")
    print(f"CER (Character Error Rate) : {overall_cer:.3f}")

    return {
        "num_sentences": len(references),
        "WER": overall_wer,
        "CER": overall_cer
    }


# Fast Whisper 71K audios

In [3]:
metrics_small = evaluate_asr_predictions("metadata_with_predictions_small.jsonl")
metrics_medium = evaluate_asr_predictions("metadata_with_predictions_medium.jsonl")
metrics_large = evaluate_asr_predictions("metadata_with_predictions_large.jsonl")

print("Small:", metrics_small)
print("Medium:", metrics_medium)
print("Large:", metrics_large)


✅ Fichier évalué : metadata_with_predictions_small.jsonl
Nombre de phrases évaluées : 70900
WER (Word Error Rate) : 0.195
CER (Character Error Rate) : 0.087
✅ Fichier évalué : metadata_with_predictions_medium.jsonl
Nombre de phrases évaluées : 70848
WER (Word Error Rate) : 0.167
CER (Character Error Rate) : 0.077
✅ Fichier évalué : metadata_with_predictions_large.jsonl
Nombre de phrases évaluées : 70931
WER (Word Error Rate) : 0.146
CER (Character Error Rate) : 0.066
Small: {'num_sentences': 70900, 'WER': 0.1952426863288178, 'CER': 0.08723724917371074}
Medium: {'num_sentences': 70848, 'WER': 0.16669175207505274, 'CER': 0.07656037507156996}
Large: {'num_sentences': 70931, 'WER': 0.1464902743806228, 'CER': 0.06649615495507732}


# Whisper 1K audios

In [6]:
import json
from pathlib import Path
from jiwer import wer, cer

def evaluate_asr_predictions(jsonl_path: str, model_size: str = "small"):
    """
    Calcule WER et CER en comparant `raw_text` et `model_prediction_<size>` dans un fichier JSONL.
    
    Paramètres :
        jsonl_path : str - chemin vers le fichier JSONL
        model_size : str - "small", "medium" ou "large" (par défaut "small")
        
    Retourne un dictionnaire avec les métriques globales.
    """
    jsonl_path = Path(jsonl_path)
    if not jsonl_path.exists():
        raise FileNotFoundError(f"{jsonl_path} n'existe pas")

    valid_sizes = {"small", "medium", "large"}
    if model_size not in valid_sizes:
        raise ValueError(f"model_size doit être l'un de {valid_sizes}")

    prediction_key = f"model_prediction_{model_size}"

    references = []
    hypotheses = []

    with open(jsonl_path, "r", encoding="utf-8") as fin:
        for line in fin:
            item = json.loads(line)
            ref = item.get("raw_text", "").strip()
            hyp = item.get(prediction_key, "").strip()
            if ref and hyp:
                references.append(ref)
                hypotheses.append(hyp)

    if not references:
        raise ValueError("Aucune référence valide trouvée dans le fichier.")

    overall_wer = wer(references, hypotheses)
    overall_cer = cer(references, hypotheses)

    print(f"✅ Fichier évalué : {jsonl_path}")
    print(f"Nombre de phrases évaluées : {len(references)}")
    print(f"Modèle évalué : {model_size}")
    print(f"WER (Word Error Rate) : {overall_wer:.3f}")
    print(f"CER (Character Error Rate) : {overall_cer:.3f}")

    return {
        "num_sentences": len(references),
        "model_size": model_size,
        "WER": overall_wer,
        "CER": overall_cer
    }


In [8]:
metrics_small = evaluate_asr_predictions("data/voxpopuli_fr_train/train_metadata_full_small_speed_benchmark.jsonl","small")
metrics_medium = evaluate_asr_predictions("data/voxpopuli_fr_train/train_metadata_full_medium_speed_benchmark.jsonl","medium")
metrics_large = evaluate_asr_predictions("data/voxpopuli_fr_train/train_metadata_full_large_speed_benchmark.jsonl","large")

print("Small:", metrics_small)
print("Medium:", metrics_medium)
print("Large:", metrics_large)


✅ Fichier évalué : data/voxpopuli_fr_train/train_metadata_full_small_speed_benchmark.jsonl
Nombre de phrases évaluées : 960
Modèle évalué : small
WER (Word Error Rate) : 0.191
CER (Character Error Rate) : 0.079
✅ Fichier évalué : data/voxpopuli_fr_train/train_metadata_full_medium_speed_benchmark.jsonl
Nombre de phrases évaluées : 958
Modèle évalué : medium
WER (Word Error Rate) : 0.162
CER (Character Error Rate) : 0.069
✅ Fichier évalué : data/voxpopuli_fr_train/train_metadata_full_large_speed_benchmark.jsonl
Nombre de phrases évaluées : 961
Modèle évalué : large
WER (Word Error Rate) : 0.142
CER (Character Error Rate) : 0.061
Small: {'num_sentences': 960, 'model_size': 'small', 'WER': 0.19091903719912473, 'CER': 0.07902004398680396}
Medium: {'num_sentences': 958, 'model_size': 'medium', 'WER': 0.16197623514696685, 'CER': 0.06852739533401246}
Large: {'num_sentences': 961, 'model_size': 'large', 'WER': 0.14169787765293385, 'CER': 0.06093666180121512}


# Whisper fine tuned French Bofenuang 1K

In [10]:
metrics_large_bofenghuang_fr = evaluate_asr_predictions("sample_1000_whisper_french.jsonl")

✅ Fichier évalué : sample_1000_whisper_french.jsonl
Nombre de phrases évaluées : 960
WER (Word Error Rate) : 0.091
CER (Character Error Rate) : 0.045
